In [2]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow import keras
from PIL import Image
from collections import Counter

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F


In [3]:
import xmltodict
import cv2
from PIL import Image

img_width = 256
img_height = 256

def zoom_at(img, x, y, zoom):
    w, h = img.size
    zoom2 = zoom * 2
    img = img.crop((x - w / zoom2, y - h / zoom2, 
                    x + w / zoom2, y + h / zoom2))
    return img.resize((w, h), Image.LANCZOS)

def get_number_of_positives_and_negatives(past, seq):
	dir_imgs = past + str(seq) + "/imgs/"
	positives = 0
	negatives = 0

	with open(past + str(seq) + "/annotations.xml", "rb") as org_file:
		dict_file = xmltodict.parse(org_file)

	arr_plants = dict_file["annotations"]["image"]

	for file in arr_plants:
		try:
			name_img = file["@name"]
			x_min = file["box"]
			positives += 1

		except Exception as e:
			negatives += 1
	return positives, negatives


def get_arrays_from_past_and_seq(past, seq):
	x_array = []
	y_array = []
	dir_imgs = past + str(seq) + "/imgs/"

	with open(past + str(seq) + "/annotations.xml", "rb") as org_file:
		dict_file = xmltodict.parse(org_file)

	arr_plants = dict_file["annotations"]["image"]

	for file in arr_plants:
		try:
			name_img = file["@name"]
			x_min = file["box"]
			img = Image.open(dir_imgs + name_img)
			img = img.resize((img_width, img_height)) # 0.3 = 576x324 --- 0.2 = 384x216 --- 0.15 = 288x162
			x_array.append(img)
			y_array.append([1])

		except:
			name_img = file["@name"]
			img = Image.open(dir_imgs + name_img)
			img = img.resize((img_width, img_height))
			#rotated_image1 = img.rotate(180)
			#rotated_image1 = zoom_at(rotated_image1, 960, 540, 1.5)
			# rotated_image1 = rotated_image1.resize((img_width, img_height))
			# img.show()
			# rotated_image1.show()
			#x_array.append(rotated_image1)
			#y_array.append([0])
			x_array.append(img)
			y_array.append([0])

	return x_array, y_array

past_and_last_seq_0 = ("RumexWeeds/20210806_hegnstrup/seq", 17)
past_and_last_seq_1 = ("RumexWeeds/20210806_stengard/seq", 20)
past_and_last_seq_2 = ("RumexWeeds/20210807_lundholm/seq", 28)
past_and_last_seq_3 = ("RumexWeeds/20210908_lundholm/seq", 13)
past_and_last_seq_4 = ("RumexWeeds/20211006_stengard/seq", 15)

x_train = []
y_train = []

x_test = []
y_test = []

# total_pos = 0
# total_neg = 0
# for i in range(past_and_last_seq_4[1]+1):
# 	p, n = get_number_of_positives_and_negatives(past_and_last_seq_4[0], i)
# 	total_pos += p
# 	total_neg += n
# 	print(i, " = ", p, "  ", n)

# print(past_and_last_seq_4[0])
# print("Positives: ", total_pos)
# print("Negatives: ", total_neg, "\n")

for i in range(past_and_last_seq_2[1]+1):
	x_aux, y_aux = get_arrays_from_past_and_seq(past_and_last_seq_2[0], i)
	x_train.extend(x_aux)
	y_train.extend(y_aux)

for i in range(past_and_last_seq_0[1]+1):
	x_aux, y_aux = get_arrays_from_past_and_seq(past_and_last_seq_0[0], i)
	x_test.extend(x_aux)
	y_test.extend(y_aux)

x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

# 17 positivas
# 10 negativas SEQ0

print(x_train.shape)
print(x_test.shape)

print("Total treino = ", len(y_train))
print("Total teste = ", len(y_test))
# (580, 1200, 1920, 3)

(1470, 256, 256, 3)
(706, 256, 256, 3)
Total treino =  1470
Total teste =  706


In [4]:
from __future__ import print_function
from PIL import Image
import numpy as np
import keras

from tensorflow.python.keras.utils import conv_utils

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, Concatenate, Reshape, Activation
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import SGD
from pool_helper import PoolHelper
from lrn import LRN

if keras.backend.backend() == 'tensorflow':
	from keras import backend as K
	import tensorflow as tf
	# from keras.utils.conv_utils import convert_kernel

def create_googlenet(weights_path=None):
	# creates GoogLeNet a.k.a. Inception v1 (Szegedy, 2015)
	input = Input(shape=(img_width, img_height, 3))
	input_pad = ZeroPadding2D(padding=(3, 3))(input) # Adiciona zeros nas bordas
	conv1_7x7_s2 = Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), padding='valid', activation='relu', name='conv1_7x7_s2', kernel_regularizer=l2(0.0002))(input_pad) # transforma em (64, 256, 256, 3)
	conv1_zero_pad = ZeroPadding2D(padding=(1, 1))(conv1_7x7_s2) 
	pool1_helper = PoolHelper()(conv1_zero_pad)
	pool1_3x3_s2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='pool1_3x3_s2')(pool1_helper)
	pool1_norm1 = LRN(name='pool1_norm1')(pool1_3x3_s2) # Normaliza os valores

	conv2_3x3_reduce = Conv2D(64, (1,1), padding='same', activation='relu', name='conv2_3x3_reduce', kernel_regularizer=l2(0.0002))(pool1_norm1)
	conv2_3x3 = Conv2D(192, (3,3), padding='same', activation='relu', name='conv2_3x3', kernel_regularizer=l2(0.0002))(conv2_3x3_reduce)
	conv2_norm2 = LRN(name='conv2_norm2')(conv2_3x3)
	conv2_zero_pad = ZeroPadding2D(padding=(1, 1))(conv2_norm2)
	pool2_helper = PoolHelper()(conv2_zero_pad)
	pool2_3x3_s2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='pool2_3x3_s2')(pool2_helper)

	inception_3a_1x1 = Conv2D(64, (1,1), padding='same', activation='relu', name='inception_3a_1x1', kernel_regularizer=l2(0.0002))(pool2_3x3_s2)
	inception_3a_3x3_reduce = Conv2D(96, (1,1), padding='same', activation='relu', name='inception_3a_3x3_reduce', kernel_regularizer=l2(0.0002))(pool2_3x3_s2)
	inception_3a_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3_reduce)
	inception_3a_3x3 = Conv2D(128, (3,3), padding='valid', activation='relu', name='inception_3a_3x3', kernel_regularizer=l2(0.0002))(inception_3a_3x3_pad)
	inception_3a_5x5_reduce = Conv2D(16, (1,1), padding='same', activation='relu', name='inception_3a_5x5_reduce', kernel_regularizer=l2(0.0002))(pool2_3x3_s2)
	inception_3a_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5_reduce)
	inception_3a_5x5 = Conv2D(32, (5,5), padding='valid', activation='relu', name='inception_3a_5x5', kernel_regularizer=l2(0.0002))(inception_3a_5x5_pad)
	inception_3a_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_3a_pool')(pool2_3x3_s2)
	inception_3a_pool_proj = Conv2D(32, (1,1), padding='same', activation='relu', name='inception_3a_pool_proj', kernel_regularizer=l2(0.0002))(inception_3a_pool)
	inception_3a_output = Concatenate(axis=-1, name='inception_3a_output')([inception_3a_1x1,inception_3a_3x3,inception_3a_5x5,inception_3a_pool_proj])

	inception_3b_1x1 = Conv2D(128, (1,1), padding='same', activation='relu', name='inception_3b_1x1', kernel_regularizer=l2(0.0002))(inception_3a_output)
	inception_3b_3x3_reduce = Conv2D(128, (1,1), padding='same', activation='relu', name='inception_3b_3x3_reduce', kernel_regularizer=l2(0.0002))(inception_3a_output)
	inception_3b_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3_reduce)
	inception_3b_3x3 = Conv2D(192, (3,3), padding='valid', activation='relu', name='inception_3b_3x3', kernel_regularizer=l2(0.0002))(inception_3b_3x3_pad)
	inception_3b_5x5_reduce = Conv2D(32, (1,1), padding='same', activation='relu', name='inception_3b_5x5_reduce', kernel_regularizer=l2(0.0002))(inception_3a_output)
	inception_3b_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5_reduce)
	inception_3b_5x5 = Conv2D(96, (5,5), padding='valid', activation='relu', name='inception_3b_5x5', kernel_regularizer=l2(0.0002))(inception_3b_5x5_pad)
	inception_3b_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_3b_pool')(inception_3a_output)
	inception_3b_pool_proj = Conv2D(64, (1,1), padding='same', activation='relu', name='inception_3b_pool_proj', kernel_regularizer=l2(0.0002))(inception_3b_pool)
	inception_3b_output = Concatenate(axis=-1, name='inception_3b_output')([inception_3b_1x1,inception_3b_3x3,inception_3b_5x5,inception_3b_pool_proj])

	inception_3b_output_zero_pad = ZeroPadding2D(padding=(1, 1))(inception_3b_output)
	pool3_helper = PoolHelper()(inception_3b_output_zero_pad)
	pool3_3x3_s2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='pool3_3x3_s2')(pool3_helper)

	inception_4a_1x1 = Conv2D(192, (1,1), padding='same', activation='relu', name='inception_4a_1x1', kernel_regularizer=l2(0.0002))(pool3_3x3_s2)
	inception_4a_3x3_reduce = Conv2D(96, (1,1), padding='same', activation='relu', name='inception_4a_3x3_reduce', kernel_regularizer=l2(0.0002))(pool3_3x3_s2)
	inception_4a_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4a_3x3_reduce)
	inception_4a_3x3 = Conv2D(208, (3,3), padding='valid', activation='relu', name='inception_4a_3x3' ,kernel_regularizer=l2(0.0002))(inception_4a_3x3_pad)
	inception_4a_5x5_reduce = Conv2D(16, (1,1), padding='same', activation='relu', name='inception_4a_5x5_reduce', kernel_regularizer=l2(0.0002))(pool3_3x3_s2)
	inception_4a_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4a_5x5_reduce)
	inception_4a_5x5 = Conv2D(48, (5,5), padding='valid', activation='relu', name='inception_4a_5x5', kernel_regularizer=l2(0.0002))(inception_4a_5x5_pad)
	inception_4a_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_4a_pool')(pool3_3x3_s2)
	inception_4a_pool_proj = Conv2D(64, (1,1), padding='same', activation='relu', name='inception_4a_pool_proj', kernel_regularizer=l2(0.0002))(inception_4a_pool)
	inception_4a_output = Concatenate(axis=-1, name='inception_4a_output')([inception_4a_1x1,inception_4a_3x3,inception_4a_5x5,inception_4a_pool_proj])

	loss1_ave_pool = AveragePooling2D(pool_size=(5,5), strides=(3,3), name='loss1_ave_pool')(inception_4a_output)
	loss1_conv = Conv2D(128, (1,1), padding='same', activation='relu', name='loss1_conv', kernel_regularizer=l2(0.0002))(loss1_ave_pool)
	loss1_flat = Flatten()(loss1_conv)
	loss1_fc = Dense(1024, activation='relu', name='loss1_fc', kernel_regularizer=l2(0.0002))(loss1_flat)
	loss1_drop_fc = Dropout(rate=0.7)(loss1_fc)
	loss1_classifier = Dense(1000, name='loss1_classifier', kernel_regularizer=l2(0.0002))(loss1_drop_fc)
	loss1_classifier_act = Activation('softmax')(loss1_classifier)

	inception_4b_1x1 = Conv2D(160, (1,1), padding='same', activation='relu', name='inception_4b_1x1', kernel_regularizer=l2(0.0002))(inception_4a_output)
	inception_4b_3x3_reduce = Conv2D(112, (1,1), padding='same', activation='relu', name='inception_4b_3x3_reduce', kernel_regularizer=l2(0.0002))(inception_4a_output)
	inception_4b_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4b_3x3_reduce)
	inception_4b_3x3 = Conv2D(224, (3,3), padding='valid', activation='relu', name='inception_4b_3x3', kernel_regularizer=l2(0.0002))(inception_4b_3x3_pad)
	inception_4b_5x5_reduce = Conv2D(24, (1,1), padding='same', activation='relu', name='inception_4b_5x5_reduce', kernel_regularizer=l2(0.0002))(inception_4a_output)
	inception_4b_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4b_5x5_reduce)
	inception_4b_5x5 = Conv2D(64, (5,5), padding='valid', activation='relu', name='inception_4b_5x5', kernel_regularizer=l2(0.0002))(inception_4b_5x5_pad)
	inception_4b_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_4b_pool')(inception_4a_output)
	inception_4b_pool_proj = Conv2D(64, (1,1), padding='same', activation='relu', name='inception_4b_pool_proj', kernel_regularizer=l2(0.0002))(inception_4b_pool)
	inception_4b_output = Concatenate(axis=-1, name='inception_4b_output')([inception_4b_1x1,inception_4b_3x3,inception_4b_5x5,inception_4b_pool_proj])

	inception_4c_1x1 = Conv2D(128, (1,1), padding='same', activation='relu', name='inception_4c_1x1', kernel_regularizer=l2(0.0002))(inception_4b_output)
	inception_4c_3x3_reduce = Conv2D(128, (1,1), padding='same', activation='relu', name='inception_4c_3x3_reduce', kernel_regularizer=l2(0.0002))(inception_4b_output)
	inception_4c_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4c_3x3_reduce)
	inception_4c_3x3 = Conv2D(256, (3,3), padding='valid', activation='relu', name='inception_4c_3x3', kernel_regularizer=l2(0.0002))(inception_4c_3x3_pad)
	inception_4c_5x5_reduce = Conv2D(24, (1,1), padding='same', activation='relu', name='inception_4c_5x5_reduce', kernel_regularizer=l2(0.0002))(inception_4b_output)
	inception_4c_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4c_5x5_reduce)
	inception_4c_5x5 = Conv2D(64, (5,5), padding='valid', activation='relu', name='inception_4c_5x5', kernel_regularizer=l2(0.0002))(inception_4c_5x5_pad)
	inception_4c_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_4c_pool')(inception_4b_output)
	inception_4c_pool_proj = Conv2D(64, (1,1), padding='same', activation='relu', name='inception_4c_pool_proj', kernel_regularizer=l2(0.0002))(inception_4c_pool)
	inception_4c_output = Concatenate(axis=-1, name='inception_4c_output')([inception_4c_1x1,inception_4c_3x3,inception_4c_5x5,inception_4c_pool_proj])

	inception_4d_1x1 = Conv2D(112, (1,1), padding='same', activation='relu', name='inception_4d_1x1', kernel_regularizer=l2(0.0002))(inception_4c_output)
	inception_4d_3x3_reduce = Conv2D(144, (1,1), padding='same', activation='relu', name='inception_4d_3x3_reduce', kernel_regularizer=l2(0.0002))(inception_4c_output)
	inception_4d_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4d_3x3_reduce)
	inception_4d_3x3 = Conv2D(288, (3,3), padding='valid', activation='relu', name='inception_4d_3x3', kernel_regularizer=l2(0.0002))(inception_4d_3x3_pad)
	inception_4d_5x5_reduce = Conv2D(32, (1,1), padding='same', activation='relu', name='inception_4d_5x5_reduce', kernel_regularizer=l2(0.0002))(inception_4c_output)
	inception_4d_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4d_5x5_reduce)
	inception_4d_5x5 = Conv2D(64, (5,5), padding='valid', activation='relu', name='inception_4d_5x5', kernel_regularizer=l2(0.0002))(inception_4d_5x5_pad)
	inception_4d_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_4d_pool')(inception_4c_output)
	inception_4d_pool_proj = Conv2D(64, (1,1), padding='same', activation='relu', name='inception_4d_pool_proj', kernel_regularizer=l2(0.0002))(inception_4d_pool)
	inception_4d_output = Concatenate(axis=-1, name='inception_4d_output')([inception_4d_1x1,inception_4d_3x3,inception_4d_5x5,inception_4d_pool_proj])

	loss2_ave_pool = AveragePooling2D(pool_size=(5,5), strides=(3,3), name='loss2_ave_pool')(inception_4d_output)
	loss2_conv = Conv2D(128, (1,1), padding='same', activation='relu', name='loss2_conv', kernel_regularizer=l2(0.0002))(loss2_ave_pool)
	loss2_flat = Flatten()(loss2_conv)
	loss2_fc = Dense(1024, activation='relu', name='loss2_fc', kernel_regularizer=l2(0.0002))(loss2_flat)
	loss2_drop_fc = Dropout(rate=0.7)(loss2_fc)
	loss2_classifier = Dense(1000, name='loss2_classifier', kernel_regularizer=l2(0.0002))(loss2_drop_fc)
	loss2_classifier_act = Activation('softmax')(loss2_classifier)

	inception_4e_1x1 = Conv2D(256, (1,1), padding='same', activation='relu', name='inception_4e_1x1', kernel_regularizer=l2(0.0002))(inception_4d_output)
	inception_4e_3x3_reduce = Conv2D(160, (1,1), padding='same', activation='relu', name='inception_4e_3x3_reduce', kernel_regularizer=l2(0.0002))(inception_4d_output)
	inception_4e_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4e_3x3_reduce)
	inception_4e_3x3 = Conv2D(320, (3,3), padding='valid', activation='relu', name='inception_4e_3x3', kernel_regularizer=l2(0.0002))(inception_4e_3x3_pad)
	inception_4e_5x5_reduce = Conv2D(32, (1,1), padding='same', activation='relu', name='inception_4e_5x5_reduce', kernel_regularizer=l2(0.0002))(inception_4d_output)
	inception_4e_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4e_5x5_reduce)
	inception_4e_5x5 = Conv2D(128, (5,5), padding='valid', activation='relu', name='inception_4e_5x5', kernel_regularizer=l2(0.0002))(inception_4e_5x5_pad)
	inception_4e_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_4e_pool')(inception_4d_output)
	inception_4e_pool_proj = Conv2D(128, (1,1), padding='same', activation='relu', name='inception_4e_pool_proj', kernel_regularizer=l2(0.0002))(inception_4e_pool)
	inception_4e_output = Concatenate(axis=-1, name='inception_4e_output')([inception_4e_1x1,inception_4e_3x3,inception_4e_5x5,inception_4e_pool_proj])

	inception_4e_output_zero_pad = ZeroPadding2D(padding=(1, 1))(inception_4e_output)
	pool4_helper = PoolHelper()(inception_4e_output_zero_pad)
	pool4_3x3_s2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='pool4_3x3_s2')(pool4_helper)

	inception_5a_1x1 = Conv2D(256, (1,1), padding='same', activation='relu', name='inception_5a_1x1', kernel_regularizer=l2(0.0002))(pool4_3x3_s2)
	inception_5a_3x3_reduce = Conv2D(160, (1,1), padding='same', activation='relu', name='inception_5a_3x3_reduce', kernel_regularizer=l2(0.0002))(pool4_3x3_s2)
	inception_5a_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_5a_3x3_reduce)
	inception_5a_3x3 = Conv2D(320, (3,3), padding='valid', activation='relu', name='inception_5a_3x3', kernel_regularizer=l2(0.0002))(inception_5a_3x3_pad)
	inception_5a_5x5_reduce = Conv2D(32, (1,1), padding='same', activation='relu', name='inception_5a_5x5_reduce', kernel_regularizer=l2(0.0002))(pool4_3x3_s2)
	inception_5a_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_5a_5x5_reduce)
	inception_5a_5x5 = Conv2D(128, (5,5), padding='valid', activation='relu', name='inception_5a_5x5', kernel_regularizer=l2(0.0002))(inception_5a_5x5_pad)
	inception_5a_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_5a_pool')(pool4_3x3_s2)
	inception_5a_pool_proj = Conv2D(128, (1,1), padding='same', activation='relu', name='inception_5a_pool_proj', kernel_regularizer=l2(0.0002))(inception_5a_pool)
	inception_5a_output = Concatenate(axis=-1, name='inception_5a_output')([inception_5a_1x1,inception_5a_3x3,inception_5a_5x5,inception_5a_pool_proj])

	inception_5b_1x1 = Conv2D(384, (1,1), padding='same', activation='relu', name='inception_5b_1x1', kernel_regularizer=l2(0.0002))(inception_5a_output)
	inception_5b_3x3_reduce = Conv2D(192, (1,1), padding='same', activation='relu', name='inception_5b_3x3_reduce', kernel_regularizer=l2(0.0002))(inception_5a_output)
	inception_5b_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_5b_3x3_reduce)
	inception_5b_3x3 = Conv2D(384, (3,3), padding='valid', activation='relu', name='inception_5b_3x3', kernel_regularizer=l2(0.0002))(inception_5b_3x3_pad)
	inception_5b_5x5_reduce = Conv2D(48, (1,1), padding='same', activation='relu', name='inception_5b_5x5_reduce', kernel_regularizer=l2(0.0002))(inception_5a_output)
	inception_5b_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_5b_5x5_reduce)
	inception_5b_5x5 = Conv2D(128, (5,5), padding='valid', activation='relu', name='inception_5b_5x5', kernel_regularizer=l2(0.0002))(inception_5b_5x5_pad)
	inception_5b_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_5b_pool')(inception_5a_output)
	inception_5b_pool_proj = Conv2D(128, (1,1), padding='same', activation='relu', name='inception_5b_pool_proj', kernel_regularizer=l2(0.0002))(inception_5b_pool)
	inception_5b_output = Concatenate(axis=-1, name='inception_5b_output')([inception_5b_1x1,inception_5b_3x3,inception_5b_5x5,inception_5b_pool_proj])

	pool5_7x7_s1 = AveragePooling2D(pool_size=(7,7), strides=(1,1), name='pool5_7x7_s2')(inception_5b_output)
	loss3_flat = Flatten()(pool5_7x7_s1)
	pool5_drop_7x7_s1 = Dropout(rate=0.4)(loss3_flat)
	loss3_classifier = Dense(1000, name='loss3_classifier', kernel_regularizer=l2(0.0002))(pool5_drop_7x7_s1)
	loss3_classifier_act = Activation('softmax', name='prob')(loss3_classifier)
	
	googlenet = Model(inputs=input, outputs=[loss1_classifier_act,loss2_classifier_act,loss3_classifier_act])

	if weights_path:
		googlenet.load_weights(weights_path)

	return googlenet

In [5]:
model = create_googlenet()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', 'accuracy', 'accuracy'])
model.fit(x_train, y_train, epochs=10)
aux = model.evaluate(x_test,y_test)

print("Acurácia: ", aux)


Epoch 1/10


C:\Users\tadio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\optimizers\base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


46/46 ━━━━━━━━━━━━━━━━━━━━ 153s 3s/step - activation_accuracy: 0.4631 - loss: 4.4022
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 135s 3s/step - activation_accuracy: 0.4917 - loss: 1.4076
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - activation_accuracy: 0.5083 - loss: 1.2107
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 136s 3s/step - activation_accuracy: 0.5095 - loss: 1.1485
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - activation_accuracy: 0.5365 - loss: 1.1142
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 138s 3s/step - activation_accuracy: 0.5072 - loss: 1.1014
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 134s 3s/step - activation_accuracy: 0.5228 - loss: 1.0723
Epoch 8/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 134s 3s/step - activation_accuracy: 0.5613 - loss: 1.0427
Epoch 9/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 134s 3s/step - activation_accuracy: 0.5697 - loss: 1.0354
Epoch 10/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 134s 3s/step - activation_accuracy: 0.5556 - loss: 1.0398
23/23 ━━━━━━━━━━━━━━━━━━━━ 19s 741ms/step - activa

In [6]:
aux = model.evaluate(x_test,y_test)

print("Acurácia: ", aux)

23/23 ━━━━━━━━━━━━━━━━━━━━ 17s 741ms/step - activation_accuracy: 0.5988 - loss: 0.9913
Acurácia:  [0.9591845273971558, 0.643059492111206]
